In [4]:
# %pip install nbformat

In [1]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [2]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_986713/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [3]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

def train_SVM_classifier(X_train, y_train, X_test, cat_cols):

    all_cols = X_train.columns.tolist()

    # Convert column names to positional indices
    cat_indices = [all_cols.index(col) for col in cat_cols]

    # Compute class weights for imbalanced training set
    class_labels = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)
    class_weight_dict = dict(zip(class_labels, class_weights))
    print("Class weights:", class_weight_dict)


        # Preprocessing for categorical features: Imputation (if needed) + One-Hot Encoding
    categorical_transformer = Pipeline(steps=[
        #   ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values (if any)
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encode categorical features
    ])

    # Combine both transformations into a single ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, cat_indices)
        ])

    # Create a pipeline that first transforms the data and then applies Logistic Regression
    pipeline = Pipeline(steps=[
        #  ('smote', SMOTENC(categorical_features=cat_cols, random_state=42, sampling_strategy=sampling_strategy)),
        ('preprocessor', preprocessor),
        ('linsvc', LinearSVC(class_weight=class_weights, dual=False))
    ])


    # Set up parameter distributions for XGBoost.
    param_distributions = {
    'linsvc__C': [0.01, 0.1, 1, 2, 4, 8, 10],
    'linsvc__loss': ['squared_hinge', 'hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
   # 'linsvc__penalty': ['l2'],  # 'l1' can only be used with dual=False and 'liblinear' solver
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
    }
    
    # Perform hyperparameter search using RandomizedSearchCV.
    randomized_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=param_distributions,
        n_iter=3,
        cv=skf, 
        scoring=balanced_acc_scorer, 
        verbose=3,
    )


        
    randomized_search.fit(X_train, y_train)
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    # Make predictions on the test set using the best estimator.
    test_predictions = randomized_search.predict(X_test)
        
    return randomized_search.best_estimator_, test_predictions


0    Return to Owner
1    Return to Owner
2           Transfer
3    Return to Owner
4    Return to Owner
Name: outcome_type, dtype: object

In [ ]:
'''

# Cross Validation with just regular SVC
from sklearn.svm import SVC

# Cross Validation
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

class_weights_cv = get_class_weights(y_train)

pipeline = Pipeline([
    ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 9], random_state=42)),
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True)),
    ('svc', SVC(class_weight=class_weights_cv, probability=False))  # Set to True if you need proba
])

# Hyperparameters for SVC
param_distributions = {
    'svc__C': [0.01, 0.1, 1, 2, 4, 7, 8 , 9, 10],
    'svc__kernel': ['rbf', 'poly', 'sigmoid', 'precomputed'],  # Try a few kernels
    'svc__gamma': ['scale', 'auto'],
    'svc__degree': [2, 3],  # Only relevant for poly kernel
    'svc__max_iter': [1000, 5000, 10000, 25000]
}

# Randomized Search
randomized_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=skf,
    scoring=balanced_acc_scorer,
    verbose=3
)

# Fit
randomized_search.fit(X_train, y_train)

# Output
print('Best parameters:', randomized_search.best_params_)
print('Best cross-validation accuracy:', randomized_search.best_score_)

cv_scores = cross_val_score(
    randomized_search.best_estimator_, X_train, y_train,
    cv=skf, verbose=3, scoring=balanced_acc_scorer
)
print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

# Predict
test_predictions = randomized_search.predict(X_test)
classification_report_with_accuracy_score(y_test, test_predictions)
'''


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [73]:
X_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,is_mix,primary_color,intake_year,is_weekend,hour_sin,hour_cos,month_sin,month_cos
107615,Stray,Normal / Behavior,Cat,Intact Female,0.083333,Domestic Shorthair,0,torbie,2021,0,0.500000,-8.660254e-01,-5.000000e-01,-0.866025
13002,Stray,Injured,Dog,Intact Male,2.000000,Akita/Australian Cattle Dog,0,brown brindle,2014,0,0.258819,-9.659258e-01,-8.660254e-01,-0.500000
32513,Public Assist,Normal / Behavior,Dog,Intact Female,1.000000,Rat Terrier,1,brown,2019,0,-0.707107,7.071068e-01,-5.000000e-01,-0.866025
79497,Stray,Normal / Behavior,Cat,Intact Female,0.083333,Domestic Shorthair,1,calico,2015,1,-1.000000,-1.836970e-16,1.224647e-16,-1.000000
88489,Stray,Normal / Behavior,Dog,Intact Female,0.333333,Labrador Retriever,0,black,2017,0,0.500000,-8.660254e-01,5.000000e-01,-0.866025
